In [ ]:
'''
Initial code was copied from:
https://github.com/jason71995/Keras-GAN-Library
'''

In [44]:
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D,GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose,Activation,BatchNormalization
from keras.optimizers import Adam

from keras.layers import Dense
from keras import initializers
from keras.layers.core import Dropout



noise_dim = 5

def build_generator(input_shape):

    generator = Sequential()
    
    generator.add(Dense(256,
                        input_dim = noise_dim))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
#     generator.add(Dense(256))
#     generator.add(BatchNormalization())
#     generator.add(LeakyReLU(0.2))
    

    


    
    generator.add(Dense(105, activation='tanh'))
    return generator


def build_discriminator(input_shape):

    discriminator = Sequential()
    discriminator.add(Dense(256,
                            input_dim=105))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    
#     discriminator.add(Dense(256))
#     discriminator.add(LeakyReLU(0.2))
#     discriminator.add(Dropout(0.3))
    

        

    
    discriminator.add(Dense(1, activation='sigmoid'))

    return discriminator

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)

    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    pred_real = K.clip(pred_real,K.epsilon(),1-K.epsilon())
    pred_fake = K.clip(pred_fake,K.epsilon(),1-K.epsilon())

    d_loss = -(K.mean(K.log(pred_real)) + K.mean(K.log(1-pred_fake)))
    g_loss = -K.mean(K.log(pred_fake))

    # get updates of mean and variance in batch normalization layers
    d_updates = discriminator.get_updates_for([d_input])
    g_updates = generator.get_updates_for([noise])

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss],d_updates + d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_updates + g_training_updates)

    return d_train,g_train

In [3]:
# Loading MHAD data for action1, all persons and all repeatations of each person
from utils.data_loader import data_loader
data_object= data_loader(matlab_action_path='../gan/')
myData, mymin, mymax = data_object.actions_normalised([1], twoD_true_or_threeD_false=False)
myData.shape

(117562, 105)

In [46]:
import keras

#from gan_libs.DCGAN import build_generator, build_discriminator, build_functions
# from gan_libs.LSGAN import build_generator, build_discriminator, build_functions
# from gan_libs.SNGAN import build_generator, build_discriminator, build_functions
# from gan_libs.WGAN_GP import build_generator, build_discriminator, build_functions

from utils.common import set_gpu_config, predict_images
from utils.draw_pose import draw_pose
import numpy as np

#set_gpu_config("0",0.5)

epoch = 35 + 1
image_size = (1,1,105)
noise_size = (1,1,5)
batch_size = 16

x_train = myData
np.random.shuffle(x_train)

generator = build_generator(noise_size)
print('gen params:',generator.count_params())
discriminator = build_discriminator(image_size)
print('dis params:',discriminator.count_params())
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

# generator.load_weights("e25_generator.h5".format(e))
# discriminator.load_weights("e25_discriminator.h5".format(e))


number_of_all_data = x_train.shape[0]
number_of_batches = int(number_of_all_data/batch_size)
print('Number of Batches passed in each epoch: ',number_of_batches)

import time
start_time = time.time()
is_first_epoch = True

for e in range(epoch):
    index = 0
    for batch in range(number_of_batches):
        real_images = x_train[index:index+batch_size]
        index =+ batch_size
        real_images.shape = (batch_size,1,1,105)
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
        if np.random.randint(low = 0, high = 100) == 1:   
            print ("[{0}/{1}] d_loss: {2:.4}, g_loss: {3:.4}".format(e, epoch, d_loss, g_loss))
            #generating a sample
            image = generator.predict(np.zeros(shape=(1,5)))
            image = np.array(image)
            draw_pose(image.reshape(105),'output',"e{0}_batch{1}".format(e,batch))
    
    if(is_first_epoch):
        elapsed_time = time.time() - start_time 
        print('\n\nTime Taken for single epoch:')
        print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
        is_first_epoch = False
    
    if e % 30 == 0 and e > 0:
        generator.save_weights("e{0}_generator.h5".format(e))
        discriminator.save_weights("e{0}_discriminator.h5".format(e))
 
#just monitoring:
##########################################################################################
elapsed_time = time.time() - start_time 
print('\n\n\n\nNumber of parameter for the Generator and discriminator respectively:\n')
print(generator.count_params())
print('')
print(discriminator.count_params())
print('\n\nNumber of Epochs and steps for each epoch:\n')
print('epochs: ',epoch, '   batches: ', number_of_batches)

print('\n\nTime Taken:')
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
##########################################################################################
#NOTE: previously in each epoch 1000 steps were iterated. in each iteration data was permitated and a batch was chosen. 
#now data is shuffled first and there are number_of_data/batch_size steps to be passed in a single epoch. 
# since 200 epochs and 1000 steps each was used, now 25 epochs and ˜7300 steps is used so the balance is kept.

gen params: 29545
dis params: 27393
Number of Batches passed in each epoch:  7347
[0/36] d_loss: 1.133, g_loss: 0.5121
[0/36] d_loss: 1.178, g_loss: 0.7518
[0/36] d_loss: 1.15, g_loss: 0.8243
[0/36] d_loss: 1.108, g_loss: 0.7763
[0/36] d_loss: 1.099, g_loss: 0.7986
[0/36] d_loss: 1.209, g_loss: 1.102
[0/36] d_loss: 1.213, g_loss: 1.044
[0/36] d_loss: 1.213, g_loss: 0.9862
[0/36] d_loss: 1.196, g_loss: 1.147
[0/36] d_loss: 1.166, g_loss: 1.02
[0/36] d_loss: 1.125, g_loss: 0.9747
[0/36] d_loss: 1.146, g_loss: 0.9706
[0/36] d_loss: 1.149, g_loss: 0.9818
[0/36] d_loss: 1.182, g_loss: 0.8997
[0/36] d_loss: 1.137, g_loss: 0.8921
[0/36] d_loss: 1.159, g_loss: 0.9675
[0/36] d_loss: 1.379, g_loss: 0.6095
[0/36] d_loss: 1.4, g_loss: 0.55
[0/36] d_loss: 1.216, g_loss: 0.6885
[0/36] d_loss: 1.105, g_loss: 0.7869
[0/36] d_loss: 1.086, g_loss: 0.845
[0/36] d_loss: 1.147, g_loss: 0.8448
[0/36] d_loss: 1.134, g_loss: 0.8362
[0/36] d_loss: 1.114, g_loss: 0.8804
[0/36] d_loss: 1.163, g_loss: 0.935
[0/36

[2/36] d_loss: 1.143, g_loss: 0.8615
[2/36] d_loss: 1.134, g_loss: 0.8765
[2/36] d_loss: 1.172, g_loss: 0.9805
[2/36] d_loss: 1.178, g_loss: 0.8452
[2/36] d_loss: 1.125, g_loss: 0.9254
[2/36] d_loss: 1.098, g_loss: 0.821
[3/36] d_loss: 1.116, g_loss: 0.823
[3/36] d_loss: 1.155, g_loss: 0.7898
[3/36] d_loss: 1.141, g_loss: 0.9395
[3/36] d_loss: 1.104, g_loss: 0.888
[3/36] d_loss: 1.107, g_loss: 0.823
[3/36] d_loss: 1.111, g_loss: 0.9115
[3/36] d_loss: 1.087, g_loss: 0.9204
[3/36] d_loss: 1.125, g_loss: 0.9292
[3/36] d_loss: 1.15, g_loss: 0.8463
[3/36] d_loss: 1.107, g_loss: 0.7869
[3/36] d_loss: 1.185, g_loss: 0.8581
[3/36] d_loss: 1.163, g_loss: 0.8757
[3/36] d_loss: 1.114, g_loss: 0.9427
[3/36] d_loss: 1.152, g_loss: 0.8772
[3/36] d_loss: 1.095, g_loss: 0.8317
[3/36] d_loss: 1.147, g_loss: 0.8735
[3/36] d_loss: 1.104, g_loss: 1.02
[3/36] d_loss: 1.191, g_loss: 0.8504
[3/36] d_loss: 1.107, g_loss: 0.9288
[3/36] d_loss: 1.042, g_loss: 0.8608
[3/36] d_loss: 1.146, g_loss: 0.9738
[3/36] d

[6/36] d_loss: 1.009, g_loss: 1.187
[6/36] d_loss: 0.9771, g_loss: 1.045
[6/36] d_loss: 1.066, g_loss: 1.19
[6/36] d_loss: 0.9021, g_loss: 1.206
[6/36] d_loss: 0.941, g_loss: 0.8923
[6/36] d_loss: 1.085, g_loss: 1.38
[6/36] d_loss: 1.016, g_loss: 1.117
[6/36] d_loss: 1.091, g_loss: 1.176
[6/36] d_loss: 1.055, g_loss: 1.289
[6/36] d_loss: 0.9644, g_loss: 1.143
[6/36] d_loss: 1.097, g_loss: 0.8175
[6/36] d_loss: 0.8908, g_loss: 0.9524
[6/36] d_loss: 0.9367, g_loss: 1.167
[6/36] d_loss: 0.9304, g_loss: 1.207
[6/36] d_loss: 1.046, g_loss: 1.091
[6/36] d_loss: 0.9527, g_loss: 0.984
[6/36] d_loss: 1.005, g_loss: 1.15
[6/36] d_loss: 1.018, g_loss: 1.099
[6/36] d_loss: 1.166, g_loss: 1.027
[6/36] d_loss: 0.9831, g_loss: 0.9933
[6/36] d_loss: 0.9439, g_loss: 1.46
[6/36] d_loss: 0.8542, g_loss: 1.265
[6/36] d_loss: 0.9191, g_loss: 1.084
[6/36] d_loss: 1.006, g_loss: 0.8989
[6/36] d_loss: 1.003, g_loss: 1.227
[6/36] d_loss: 0.9298, g_loss: 1.2
[6/36] d_loss: 1.01, g_loss: 1.168
[6/36] d_loss: 0.8

[9/36] d_loss: 0.7536, g_loss: 1.182
[9/36] d_loss: 0.604, g_loss: 1.834
[9/36] d_loss: 0.802, g_loss: 1.614
[9/36] d_loss: 0.8716, g_loss: 1.512
[9/36] d_loss: 0.863, g_loss: 1.627
[9/36] d_loss: 0.8873, g_loss: 1.531
[9/36] d_loss: 0.83, g_loss: 1.595
[9/36] d_loss: 0.7954, g_loss: 1.619
[9/36] d_loss: 0.7963, g_loss: 1.232
[9/36] d_loss: 0.7479, g_loss: 1.619
[9/36] d_loss: 0.7197, g_loss: 1.506
[9/36] d_loss: 0.9267, g_loss: 1.634
[9/36] d_loss: 0.9403, g_loss: 1.605
[9/36] d_loss: 0.8335, g_loss: 1.758
[9/36] d_loss: 0.7845, g_loss: 1.723
[9/36] d_loss: 0.8878, g_loss: 1.878
[9/36] d_loss: 0.8204, g_loss: 1.167
[9/36] d_loss: 0.6185, g_loss: 2.313
[9/36] d_loss: 0.869, g_loss: 1.302
[9/36] d_loss: 0.7318, g_loss: 1.632
[9/36] d_loss: 0.6728, g_loss: 1.502
[9/36] d_loss: 0.8349, g_loss: 1.164
[9/36] d_loss: 0.8546, g_loss: 1.386
[9/36] d_loss: 0.9042, g_loss: 1.9
[9/36] d_loss: 0.8354, g_loss: 1.924
[9/36] d_loss: 0.9644, g_loss: 0.8755
[9/36] d_loss: 0.754, g_loss: 1.932
[9/36] d_

[12/36] d_loss: 0.608, g_loss: 2.539
[12/36] d_loss: 0.637, g_loss: 2.498
[12/36] d_loss: 0.7125, g_loss: 1.987
[12/36] d_loss: 0.7523, g_loss: 2.171
[12/36] d_loss: 1.213, g_loss: 1.246
[12/36] d_loss: 0.6674, g_loss: 2.162
[12/36] d_loss: 0.6207, g_loss: 2.163
[12/36] d_loss: 0.5134, g_loss: 2.019
[12/36] d_loss: 0.7434, g_loss: 1.87
[12/36] d_loss: 0.8246, g_loss: 2.045
[12/36] d_loss: 0.7431, g_loss: 2.518
[12/36] d_loss: 1.004, g_loss: 1.434
[12/36] d_loss: 0.5067, g_loss: 2.611
[12/36] d_loss: 0.525, g_loss: 2.554
[12/36] d_loss: 0.6376, g_loss: 2.581
[12/36] d_loss: 0.6097, g_loss: 1.606
[12/36] d_loss: 0.7794, g_loss: 2.47
[12/36] d_loss: 0.7186, g_loss: 2.172
[12/36] d_loss: 0.5956, g_loss: 1.584
[12/36] d_loss: 0.5354, g_loss: 2.297
[12/36] d_loss: 0.6021, g_loss: 2.858
[12/36] d_loss: 0.5209, g_loss: 2.005
[12/36] d_loss: 0.8034, g_loss: 2.475
[12/36] d_loss: 0.5527, g_loss: 2.226
[12/36] d_loss: 0.7349, g_loss: 2.067
[12/36] d_loss: 0.7191, g_loss: 1.423
[12/36] d_loss: 0.6

...

[22/36] d_loss: 0.3621, g_loss: 3.872
[22/36] d_loss: 0.3628, g_loss: 4.567
[22/36] d_loss: 0.3763, g_loss: 3.049
[22/36] d_loss: 0.4343, g_loss: 2.837
[22/36] d_loss: 0.4738, g_loss: 3.737
[22/36] d_loss: 0.4225, g_loss: 4.154
[22/36] d_loss: 0.4088, g_loss: 2.881
[22/36] d_loss: 1.277, g_loss: 2.585
[22/36] d_loss: 0.4878, g_loss: 3.59
[22/36] d_loss: 0.5633, g_loss: 3.122
[22/36] d_loss: 0.7256, g_loss: 3.174
[22/36] d_loss: 0.9184, g_loss: 2.578
[22/36] d_loss: 0.6068, g_loss: 2.942
[22/36] d_loss: 0.7694, g_loss: 2.774
[22/36] d_loss: 0.4481, g_loss: 2.798
[22/36] d_loss: 0.4682, g_loss: 4.81
[22/36] d_loss: 0.6544, g_loss: 3.557
[22/36] d_loss: 0.5196, g_loss: 2.329
[22/36] d_loss: 0.416, g_loss: 2.845
[22/36] d_loss: 0.9213, g_loss: 3.904
[22/36] d_loss: 0.5745, g_loss: 3.131
[22/36] d_loss: 0.5311, g_loss: 2.985
[22/36] d_loss: 0.5172, g_loss: 2.757
[22/36] d_loss: 0.6078, g_loss: 3.352
[22/36] d_loss: 0.2645, g_loss: 3.711
[22/36] d_loss: 0.5541, g_loss: 3.691
[22/36] d_loss: 

[25/36] d_loss: 0.4946, g_loss: 3.198
[25/36] d_loss: 0.7169, g_loss: 1.57
[25/36] d_loss: 0.6317, g_loss: 2.629
[25/36] d_loss: 0.4075, g_loss: 3.126
[25/36] d_loss: 0.3986, g_loss: 4.189
[25/36] d_loss: 0.6205, g_loss: 1.938
[25/36] d_loss: 0.5345, g_loss: 2.167
[25/36] d_loss: 0.5187, g_loss: 3.135
[25/36] d_loss: 0.7786, g_loss: 1.883
[25/36] d_loss: 0.2733, g_loss: 2.184
[25/36] d_loss: 0.9889, g_loss: 2.385
[25/36] d_loss: 0.5314, g_loss: 3.15
[25/36] d_loss: 0.8895, g_loss: 1.419
[25/36] d_loss: 0.6342, g_loss: 3.625
[25/36] d_loss: 0.5651, g_loss: 3.795
[25/36] d_loss: 1.116, g_loss: 1.742
[25/36] d_loss: 0.4068, g_loss: 1.741
[25/36] d_loss: 0.7351, g_loss: 3.307
[25/36] d_loss: 0.4791, g_loss: 2.666
[25/36] d_loss: 0.5843, g_loss: 2.738
[25/36] d_loss: 0.2897, g_loss: 3.718
[25/36] d_loss: 0.4585, g_loss: 1.879
[25/36] d_loss: 0.4939, g_loss: 3.655
[25/36] d_loss: 0.6638, g_loss: 1.825
[25/36] d_loss: 0.5748, g_loss: 3.836
[25/36] d_loss: 0.404, g_loss: 2.686
[25/36] d_loss: 

[28/36] d_loss: 0.7494, g_loss: 4.097
[28/36] d_loss: 0.3857, g_loss: 2.801
[28/36] d_loss: 0.5845, g_loss: 3.25
[28/36] d_loss: 0.4784, g_loss: 2.589
[28/36] d_loss: 0.7987, g_loss: 3.983
[28/36] d_loss: 0.4405, g_loss: 2.621
[28/36] d_loss: 0.524, g_loss: 3.909
[28/36] d_loss: 0.3578, g_loss: 3.429
[28/36] d_loss: 0.476, g_loss: 3.124
[28/36] d_loss: 0.3839, g_loss: 4.127
[28/36] d_loss: 0.5207, g_loss: 3.845
[28/36] d_loss: 0.4475, g_loss: 2.57
[28/36] d_loss: 0.9057, g_loss: 2.506
[28/36] d_loss: 0.5374, g_loss: 2.42
[28/36] d_loss: 0.4865, g_loss: 3.602
[28/36] d_loss: 0.4825, g_loss: 2.642
[28/36] d_loss: 0.4276, g_loss: 1.905
[28/36] d_loss: 0.5096, g_loss: 2.7
[28/36] d_loss: 0.4766, g_loss: 1.8
[28/36] d_loss: 1.156, g_loss: 0.842
[28/36] d_loss: 0.6001, g_loss: 2.328
[28/36] d_loss: 0.5274, g_loss: 2.153
[28/36] d_loss: 1.131, g_loss: 2.909
[28/36] d_loss: 0.6966, g_loss: 2.757
[28/36] d_loss: 0.5308, g_loss: 2.589
[28/36] d_loss: 0.7527, g_loss: 3.112
[28/36] d_loss: 0.5615,

[31/36] d_loss: 0.6491, g_loss: 1.318
[31/36] d_loss: 0.5001, g_loss: 2.538
[31/36] d_loss: 0.6022, g_loss: 2.954
[31/36] d_loss: 0.679, g_loss: 3.13
[31/36] d_loss: 0.415, g_loss: 2.645
[31/36] d_loss: 1.617, g_loss: 1.038
[31/36] d_loss: 0.4527, g_loss: 3.068
[31/36] d_loss: 0.6638, g_loss: 2.507
[31/36] d_loss: 0.4432, g_loss: 2.319
[31/36] d_loss: 0.5777, g_loss: 2.777
[31/36] d_loss: 0.7146, g_loss: 2.835
[31/36] d_loss: 0.5735, g_loss: 2.762
[31/36] d_loss: 0.7469, g_loss: 1.676
[31/36] d_loss: 0.5859, g_loss: 1.954
[31/36] d_loss: 0.8026, g_loss: 2.478
[31/36] d_loss: 0.4102, g_loss: 2.031
[31/36] d_loss: 0.5814, g_loss: 3.873
[31/36] d_loss: 1.464, g_loss: 2.805
[31/36] d_loss: 0.4799, g_loss: 2.464
[31/36] d_loss: 1.066, g_loss: 1.052
[31/36] d_loss: 0.4232, g_loss: 2.825
[31/36] d_loss: 0.5236, g_loss: 2.622
[31/36] d_loss: 0.5484, g_loss: 2.743
[31/36] d_loss: 0.6112, g_loss: 1.789
[31/36] d_loss: 0.7062, g_loss: 1.364
[31/36] d_loss: 0.4174, g_loss: 1.91
[31/36] d_loss: 1.1

[33/36] d_loss: 0.7788, g_loss: 2.873
[34/36] d_loss: 1.503, g_loss: 2.274
[34/36] d_loss: 0.4136, g_loss: 1.584
[34/36] d_loss: 0.3439, g_loss: 4.362
[34/36] d_loss: 0.7739, g_loss: 2.01
[34/36] d_loss: 0.4575, g_loss: 2.583
[34/36] d_loss: 0.6212, g_loss: 2.24
[34/36] d_loss: 0.766, g_loss: 2.314
[34/36] d_loss: 0.8317, g_loss: 1.556
[34/36] d_loss: 0.6043, g_loss: 1.688
[34/36] d_loss: 0.565, g_loss: 1.667
[34/36] d_loss: 0.6024, g_loss: 2.452
[34/36] d_loss: 0.7704, g_loss: 0.8015
[34/36] d_loss: 0.4671, g_loss: 3.061
[34/36] d_loss: 0.5584, g_loss: 2.095
[34/36] d_loss: 0.6283, g_loss: 1.805
[34/36] d_loss: 0.4028, g_loss: 3.062
[34/36] d_loss: 0.4741, g_loss: 2.466
[34/36] d_loss: 0.8232, g_loss: 3.719
[34/36] d_loss: 0.5943, g_loss: 1.658
[34/36] d_loss: 0.7517, g_loss: 3.265
[34/36] d_loss: 0.2462, g_loss: 3.03
[34/36] d_loss: 0.6451, g_loss: 2.623
[34/36] d_loss: 0.5276, g_loss: 2.673
[34/36] d_loss: 0.8867, g_loss: 0.5999
[34/36] d_loss: 0.4197, g_loss: 3.538
[34/36] d_loss: 

In [18]:
    #gpu details:
    ################----------------------------
from six.moves import cStringIO as StringIO
import gpustat

gpustats = gpustat.new_query()
fp = StringIO()
gpustats.print_formatted(
     fp=fp, no_color=False, show_user=False,
     show_cmd=False, show_pid=False, show_power=False, show_fan_speed=False)

result = fp.getvalue()
print('\n\n')
print(result)
    ################----------------------------




cbrc-All-Series  Mon May  6 14:40:03 2019
[0] GeForce GTX 1080 Ti | 53'C,   0 % |  6283 / 11172 MB | cbrc(5961M) gdm(16M) gdm(50M) cbrc(114M) cbrc(136M)



In [ ]:
generator.load_weights("e400_generator.h5".format(e))
discriminator.load_weights("e400_discriminator.h5".format(e))

In [ ]:
z = -1;
for i in np.linspace(-3,3,100):
    z+=1
    x= np.array((-i,0,0,i,0)).reshape(1,5)
    image = generator.predict(x)
    image = np.array(image)
    draw_pose(image.reshape(105),'output',"e{0}".format(z))